In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
import op

In [2]:
# Function to read data from the database
def reade_db():
    folder = './data/db'
    db = dict()
    for file in os.listdir(folder):
        data_field = file.split('.')[0]
        file = os.path.join(folder, file)
        print(file)
        db[data_field] = pd.read_excel(file, index_col=0, parse_dates=True)
        print(db[data_field].shape)
    return db

# Read database
db = reade_db()

# Calculate trading volume ratio (trv) and market capitalization (mktc)
db['trv'] = (db['volume'] / db['shares_outstanding']).copy()
db['mktc'] = (db['close'] * db['shares_outstanding']).copy()

./data/db/shares_outstanding.xlsx
(23, 588)
./data/db/c2c_ret.xlsx
(23, 588)
./data/db/o2o_ret.xlsx
(23, 588)
./data/db/close.xlsx
(23, 588)
./data/db/open.xlsx
(23, 588)
./data/db/volume.xlsx
(23, 588)
./data/db/univ.xlsx
(23, 588)


In [21]:
# Read open trade data
open_trade = pd.read_excel('data/open_trade.xlsx')
open_trade = open_trade.pivot(index='date', columns='ticker', values=['price', 'volume'])
open_trade_price = open_trade['price']
open_trade_volume = open_trade['volume']
open_trade_value = (open_trade_price * open_trade_volume).copy()

In [22]:
# Get the index and columns for open trade volume data
open_trade_idxcol = open_trade_volume.index, open_trade_volume.columns

# Get the index and columns for database 'univ' data
db_idxcol = db['univ'].index, db['univ'].columns

# Find the intersection of tickers in open trade and database
intersection_tickers = set.intersection(set(db_idxcol[1]), set(open_trade_idxcol[1]))
intersection_tickers = sorted(list(intersection_tickers))

In [23]:
# Calculate the ratio of open trade volume to database volume (ov2tv)
ov2tv = open_trade_volume / db['volume']
ov2tv = ov2tv.loc[open_trade_idxcol[0], intersection_tickers]
ov2tv.shape

(22, 532)

In [24]:
# Filter out abnormal tickers with ov2tv below a threshold
threshold = 0.001
count = 0 
abnormal_tickers = []
for date in ov2tv.index:
    for ticker in ov2tv.columns:
        if ov2tv.loc[date, ticker] < threshold:
            count += 1
            print(f'({count}) ', date.date(), ticker, ov2tv.loc[date, ticker])
            abnormal_tickers.append(ticker)

# Set abnormal tickers to NaN
ov2tv.loc[:, abnormal_tickers] = np.nan
open_trade_value.loc[:, abnormal_tickers] = np.nan

(1)  2024-05-01 CBOE 5.044795919320634e-05
(2)  2024-05-02 CBOE 3.478700863350305e-05
(3)  2024-05-03 CBOE 0.00023557347025780702
(4)  2024-05-03 HES 0.0009148828251980501
(5)  2024-05-06 CBOE 5.452978962407163e-05
(6)  2024-05-06 NWS 0.0008892155441916316
(7)  2024-05-07 CBOE 0.00032991771464058374
(8)  2024-05-08 CBOE 0.0002886006890341451
(9)  2024-05-09 CBOE 2.8764110757515932e-05
(10)  2024-05-10 CBOE 1.1941215103931218e-05
(11)  2024-05-13 CBOE 3.611886389755377e-05
(12)  2024-05-14 CBOE 0.00031307594159066206
(13)  2024-05-15 CBOE 2.9424486469149896e-06
(14)  2024-05-16 CBOE 0.00012081854564675678
(15)  2024-05-17 CBOE 0.0002312623309797573
(16)  2024-05-20 CBOE 0.00014325204525869305
(17)  2024-05-21 CBOE 0.0005602889519702117
(18)  2024-05-22 CBOE 0.0004303007376584074
(19)  2024-05-23 CBOE 6.875061231014089e-06
(20)  2024-05-24 CBOE 6.126962925747336e-06
(21)  2024-05-28 CBOE 4.747541564726399e-06
(22)  2024-05-28 TFX 0.0007713831277963765
(23)  2024-05-29 CBOE 7.683312594635

In [7]:
# Describe the ov2tv data
describe = ov2tv.describe()
describe

,A,AAL,AAP,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,...,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZIMV,ZION,ZTS
count,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,...,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000
mean,0.010589,0.006151,0.007806,0.015004,0.017729,0.013385,0.015586,0.010186,0.018694,0.015875,...,0.012215,0.005494,0.012065,0.012942,0.008974,0.008314,0.009611,0.012190,0.008864,0.015128
std,0.005032,0.002119,0.004797,0.009464,0.009523,0.007605,0.006812,0.005687,0.008241,0.009312,...,0.006308,0.002268,0.005571,0.005479,0.006754,0.005539,0.004256,0.009713,0.004056,0.008189
min,0.003649,0.003693,0.003432,0.008196,0.004685,0.005474,0.008077,0.001779,0.009703,0.008634,...,0.004736,0.001996,0.004332,0.003709,0.002836,0.002733,0.004006,0.002816,0.003861,0.007646
25%,0.007371,0.004593,0.004434,0.011049,0.011966,0.008564,0.010433,0.007290,0.012670,0.011077,...,0.007968,0.004248,0.008189,0.009341,0.005661,0.004836,0.007145,0.006624,0.005818,0.009323
50%,0.010105,0.005515,0.006791,0.013390,0.015746,0.010470,0.014236,0.008255,0.017104,0.014328,...,0.010639,0.005237,0.010524,0.011885,0.007359,0.006230,0.008596,0.009017,0.008106,0.011946
75%,0.011979,0.007521,0.008781,0.015499,0.019195,0.014954,0.019557,0.010718,0.022370,0.016973,...,0.015994,0.005919,0.013368,0.016176,0.010812,0.009648,0.010552,0.012362,0.011663,0.018417
max,0.025301,0.011190,0.023892,0.054824,0.046398,0.035228,0.037624,0.024588,0.045583,0.053894,...,0.033004,0.010459,0.029096,0.027700,0.036766,0.021782,0.023648,0.039227,0.019447,0.036839


In [8]:
# Calculate the median of ov2tv data
describe.median(axis=1)

count    22.000000
mean      0.010214
std       0.005347
min       0.003715
25%       0.007005
50%       0.008930
75%       0.011757
max       0.026999
dtype: float64

In [25]:
# Describe the open_trade_value data
describe = open_trade_value.describe()
describe

ticker,A,AA,AAAU,AAL,AAN,AAOI,AAON,AAP,AAPL,AAT,...,ZS,ZSL,ZTO,ZTS,ZUMZ,ZUO,ZVIA,ZWS,ZYME,ZYXI
count,2.200000e+01,2.200000e+01,22.000000,2.200000e+01,22.000000,22.000000,2.200000e+01,2.200000e+01,2.200000e+01,22.000000,...,2.200000e+01,22.000000,2.200000e+01,2.200000e+01,22.000000,22.000000,22.000000,2.200000e+01,22.000000,22.000000
mean,2.305442e+06,2.325077e+06,38234.388636,2.881433e+06,18929.474773,238754.574318,5.272591e+05,8.824560e+05,1.614234e+08,43370.521818,...,4.499986e+06,75177.546591,6.082846e+05,6.512357e+06,31547.022045,173599.971591,3906.546941,1.897172e+05,47391.885682,18062.066818
std,1.672731e+06,1.336258e+06,50829.095922,2.743552e+06,14891.458751,155074.626418,3.818080e+05,9.594814e+05,1.172232e+08,51109.497656,...,3.923823e+06,75737.834933,5.626523e+05,3.977551e+06,17233.570427,170904.318640,4961.982887,2.054861e+05,29792.336657,11671.386461
min,9.717008e+05,7.655533e+05,658.840000,1.294407e+06,6176.400000,107205.060000,2.074283e+05,3.662246e+05,7.115469e+07,7077.890000,...,1.842623e+06,6887.280000,1.140089e+05,2.876025e+06,9340.200000,55113.100000,99.736500,7.041827e+04,17954.830000,5747.210000
25%,1.366758e+06,1.447976e+06,20043.125000,1.820080e+06,9547.035000,143283.342500,2.778879e+05,4.020872e+05,1.071182e+08,22321.917500,...,2.209912e+06,25131.882500,2.764779e+05,4.162592e+06,21642.657500,81082.643750,1047.178650,1.100509e+05,25890.886250,11947.502500
50%,1.811990e+06,1.964820e+06,28297.660000,2.163045e+06,12283.722500,187709.060000,4.056609e+05,5.300586e+05,1.335967e+08,32927.025000,...,3.286512e+06,47619.990000,3.945757e+05,4.663514e+06,28446.150000,106433.905000,1939.007750,1.352206e+05,38366.410000,15343.300000
75%,2.567757e+06,2.677889e+06,42762.000000,3.027534e+06,20153.155000,252907.235000,5.769019e+05,8.131581e+05,1.651510e+08,42646.880000,...,4.451260e+06,101049.675000,6.527304e+05,6.769094e+06,31314.022500,202109.677500,4681.985625,1.690932e+05,57831.690000,20286.407500
max,7.174516e+06,5.792467e+06,255287.000000,1.456118e+07,66834.220000,735287.400000,1.567787e+06,4.268624e+06,5.835791e+08,258954.280000,...,1.746919e+07,336609.000000,2.159357e+06,1.784185e+07,90083.130000,810357.400000,22609.730000,1.069745e+06,133602.460000,57923.550000


In [26]:
# Calculate the median of open_trade_value data
describe.median(axis=1)

count        22.000000
mean     127135.808295
std      100401.822372
min       32778.510000
25%       64766.863750
50%       94097.373750
75%      141097.127500
max      463293.110000
dtype: float64

In [37]:
open_trade_value.mean(axis=0)[intersection_tickers].sort_values().dropna()

ticker
ONL     2.694309e+04
ZIMV    4.868891e+04
EMBC    7.331892e+04
FOX     1.163689e+05
LUMN    1.182406e+05
            ...     
META    7.984950e+07
AMZN    1.097567e+08
MSFT    1.176218e+08
AAPL    1.614234e+08
NVDA    3.060501e+08
Length: 526, dtype: float64

In [49]:
# Prepare market capitalization (mktc) data for regression
mktc = db['mktc'].copy()
idxcol = mktc.index, mktc.columns
mktc = np.log(mktc)
mktc = op.cs_zscore(mktc.values, db['univ'].values)
mktc = pd.DataFrame(mktc, index=idxcol[0], columns=idxcol[1])
mktc_lag = mktc.shift(1)

# Prepare trading volume ratio (trv) data for regression
trv = db['trv'].copy()
idxcol = trv.index, trv.columns
trv = np.log(trv)
trv = op.cs_zscore(trv.values, db['univ'].values)
trv = pd.DataFrame(trv, index=idxcol[0], columns=idxcol[1])
trv_lag = trv.shift(1)

# Prepare close-to-close return (c2c_ret) data for regression
c2c_ret = db['c2c_ret'].copy()
idxcol = c2c_ret.index, c2c_ret.columns
c2c_ret = np.log(c2c_ret + 1)
c2c_absret = np.fabs(c2c_ret)
c2c_absret = op.cs_zscore(c2c_absret.values, db['univ'].values)
c2c_absret = pd.DataFrame(c2c_absret, index=idxcol[0], columns=idxcol[1])
c2c_absret_lag = c2c_absret.shift(1)

In [50]:
# Combine data for regression analysis
data = pd.DataFrame({
    'mktc_lag': mktc_lag.stack(),
    'trv': trv.stack(),
    'trv_lag': trv_lag.stack(),
    'c2c_absret': c2c_absret.stack(),
    'c2c_absret_lag': c2c_absret_lag.stack(),
    'ov2tv': ov2tv.stack()
}).dropna()
data


mktc_lag       trv   trv_lag  c2c_absret  c2c_absret_lag  \
2024-05-01 A     0.037958 -1.618092 -1.469770   -0.116480        0.026039   
           AAL  -1.375710  2.656909  2.750964   -0.408734        0.972346   
           AAPL  3.950544 -1.413206 -1.138363   -0.370164        0.026229   
           ABBV  1.881070 -1.670901 -1.688471   -0.387042       -0.666029   
           ABNB  0.561886 -0.392989 -0.294630    0.027345        0.296557   
...                   ...       ...       ...         ...             ...   
2024-05-31 XYL  -0.132868 -0.317730  0.133665   -0.212969       -0.046048   
           YUM  -0.023071 -0.721324 -0.269810    0.018154       -0.695637   
           ZBH  -0.468689  0.274412  0.333930   -0.379061       -0.428184   
           ZBRA -0.808794 -0.408823 -0.595605   -0.149377       -0.711283   
           ZTS   0.640836 -0.621622 -0.074019   -0.548484       -0.323323   

                    ov2tv  
2024-05-01 A     0.012205  
           AAL   0.004587  
           AAPL  0.008481  
           ABBV  0.010478  
           ABNB  0.015073  
...                   ...  
2024-05-31 XYL   0.007844  
           YUM   0.004329  
           ZBH   0.002733  
           ZBRA  0.004006  
           ZTS   0.008997  

[10562 rows x 6 columns]

In [51]:
# Reset index for the combined data
data = data.reset_index(drop=True)
data

,mktc_lag,trv,trv_lag,c2c_absret,c2c_absret_lag,ov2tv
0,0.037958,-1.618092,-1.469770,-0.116480,0.026039,0.012205
1,-1.375710,2.656909,2.750964,-0.408734,0.972346,0.004587
2,3.950544,-1.413206,-1.138363,-0.370164,0.026229,0.008481
3,1.881070,-1.670901,-1.688471,-0.387042,-0.666029,0.010478
4,0.561886,-0.392989,-0.294630,0.027345,0.296557,0.015073
...,...,...,...,...,...,...
10557,-0.132868,-0.317730,0.133665,-0.212969,-0.046048,0.007844
10558,-0.023071,-0.721324,-0.269810,0.018154,-0.695637,0.004329
10559,-0.468689,0.274412,0.333930,-0.379061,-0.428184,0.002733
10560,-0.808794,-0.408823,-0.595605,-0.149377,-0.711283,0.004006


In [53]:
# Independent variables (X) and dependent variable (y)
X = data[['mktc_lag', 'c2c_absret_lag', 'c2c_absret', 'trv_lag', 'trv']]
y = data['ov2tv']

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Perform OLS regression
model = sm.OLS(y, X)
results = model.fit()

# Print the regression summary
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  ov2tv   R-squared:                       0.142
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     348.8
Date:                Fri, 19 Jul 2024   Prob (F-statistic):               0.00
Time:                        18:45:05   Log-Likelihood:                 37888.
No. Observations:               10562   AIC:                        -7.576e+04
Df Residuals:                   10556   BIC:                        -7.572e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.0108   6.52e-05    166.

In [54]:
# Add predicted ov2tv values to the data
data['predicted_ov2tv'] = results.predict(X)
# Display the data with predicted values
data

,mktc_lag,trv,trv_lag,c2c_absret,c2c_absret_lag,ov2tv,predicted_ov2tv
0,0.037958,-1.618092,-1.469770,-0.116480,0.026039,0.012205,0.013680
1,-1.375710,2.656909,2.750964,-0.408734,0.972346,0.004587,0.004899
2,3.950544,-1.413206,-1.138363,-0.370164,0.026229,0.008481,0.019176
3,1.881070,-1.670901,-1.688471,-0.387042,-0.666029,0.010478,0.015956
4,0.561886,-0.392989,-0.294630,0.027345,0.296557,0.015073,0.012527
...,...,...,...,...,...,...,...
10557,-0.132868,-0.317730,0.133665,-0.212969,-0.046048,0.007844,0.011722
10558,-0.023071,-0.721324,-0.269810,0.018154,-0.695637,0.004329,0.012395
10559,-0.468689,0.274412,0.333930,-0.379061,-0.428184,0.002733,0.009588
10560,-0.808794,-0.408823,-0.595605,-0.149377,-0.711283,0.004006,0.009818


In [55]:
# Calculate the average error using the mean ov2tv as the benchmark
ov2tv_mean = data['ov2tv'].mean()
avg_error = (data['ov2tv'] - ov2tv_mean).abs().mean()
avg_error /= ov2tv_mean
avg_error

np.float64(0.43620933665180756)

In [56]:
# Calculate the average error using the predicted ov2tv as the benchmark
ov2tv_mean = data['ov2tv'].mean()
avg_error = (data['ov2tv'] - data['predicted_ov2tv']).abs().mean()
avg_error /= ov2tv_mean
avg_error

np.float64(0.39268146959234046)